[View in Colaboratory](https://colab.research.google.com/github/yutinglaitw/learning-tensorflow/blob/master/2_NeuralNetworks/neural_network_layer.ipynb)

# Neural Network Example

用 tf.layer 實作多層感知機 (Multi-Layer Perceptron, MLP) 

以 MNIST 為例實作一個有兩層隱含層的 Fully Connected Network


## Load MNIST dataset

In [42]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 2-Layer Neural Network

In [0]:
# Network Parameters
h1_units = 256
h2_units = 256
num_classes = 10

# Training Parameters
learning_rate = 0.001
epochs = 1000
batch_size = 100

In [0]:
def neural_network(x):
    # Hidden layer 1
    hidden1 = tf.layers.dense(x, h1_units)

    # Hidden layer 2
    hidden2 = tf.layers.dense(hidden1, h2_units)

    # Output layer
    out_layer = tf.layers.dense(hidden2, num_classes)
    return out_layer

In [0]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    logits = neural_network(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [46]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpud5xeuqm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbf85a5cba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [47]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x=mnist.train.images, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)

# Train the Model# Train 
model.train(input_fn, steps=epochs)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpud5xeuqm/model.ckpt.
INFO:tensorflow:loss = 2.3302343, step = 0
INFO:tensorflow:global_step/sec: 390.417
INFO:tensorflow:loss = 2.0011423, step = 100 (0.259 sec)
INFO:tensorflow:global_step/sec: 407.075
INFO:tensorflow:loss = 1.7711937, step = 200 (0.247 sec)
INFO:tensorflow:global_step/sec: 445.427
INFO:tensorflow:loss = 1.587053, step = 300 (0.225 sec)
INFO:tensorflow:global_step/sec: 405.145
INFO:tensorflow:loss = 1.5098169, step = 400 (0.246 sec)
INFO:tensorflow:global_step/sec: 416.733
INFO:tensorflow:loss = 1.2850122, step = 500 (0.241 sec)
INFO:tensorflow:global_step/sec: 443.47
INFO:tensorflow:loss = 1.29993, step = 600 (0.225 sec)
INFO:tensorflow:global_step/sec: 405.957
INFO:tensorflow:loss

## Predict

In [48]:
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x=mnist.test.images, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)

# Use the Estimator 'evaluate' method
model.evaluate(input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-20-15:23:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpud5xeuqm/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-20-15:23:29
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.8136, global_step = 1000, loss = 0.9034724
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /tmp/tmpud5xeuqm/model.ckpt-1000


{'accuracy': 0.8136, 'global_step': 1000, 'loss': 0.9034724}